In [12]:
from collections import namedtuple, deque

class LSTMdata(object):
    def __init__(self, capacity):
        self.queue = deque([], maxlen=capacity)

    def push(self, data):
        self.queue.append(data)
    
    def display(self):
        print(self.queue)


In [8]:
import torch

# 创建一个3x3x5的递增张量
t = torch.arange(3*3*5).reshape(3, 3, 5)

t

tensor([[[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14]],

        [[15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24],
         [25, 26, 27, 28, 29]],

        [[30, 31, 32, 33, 34],
         [35, 36, 37, 38, 39],
         [40, 41, 42, 43, 44]]])

In [15]:
queue = LSTMdata(3)
for i in range(3):
    queue.push(t[0][i])
    queue.display()
    print('-'*20)

for i in range(1,3):
    queue.push(t[i][2])
    queue.display()
    print('-'*20)

deque([tensor([0, 1, 2, 3, 4])], maxlen=3)
--------------------
deque([tensor([0, 1, 2, 3, 4]), tensor([5, 6, 7, 8, 9])], maxlen=3)
--------------------
deque([tensor([0, 1, 2, 3, 4]), tensor([5, 6, 7, 8, 9]), tensor([10, 11, 12, 13, 14])], maxlen=3)
--------------------
deque([tensor([5, 6, 7, 8, 9]), tensor([10, 11, 12, 13, 14]), tensor([25, 26, 27, 28, 29])], maxlen=3)
--------------------
deque([tensor([10, 11, 12, 13, 14]), tensor([25, 26, 27, 28, 29]), tensor([40, 41, 42, 43, 44])], maxlen=3)
--------------------
